In [1]:
import glob
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm
import time
from skimage.color import lab2rgb, ycbcr2rgb
from fastai.vision.learner import create_body
from torchvision.models.resnet import resnet18
from fastai.vision.models.unet import DynamicUnet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from importlib import reload
import loss
import data_generator
import model
import train
import visualize
reload(loss)
reload(visualize)
reload(model)
reload(data_generator)
reload(train)

from data_generator import make_dataloaders
from model import MainModel
from train import train_model, load_model, build_res_unet, pretrain_generator

In [2]:
COLOR_SPACE = 'Lab' # Lab or HSL or YCbCr
path = "./data/part1"
model_path = f"./models/model_cropped_faces_{COLOR_SPACE}.pt"
paths = glob.glob(path + "/*.jpg") # Grabbing all the image file names
np.random.seed(123)
paths_subset = np.random.choice(paths, 1000, replace=False) # choosing 1000 images randomly
rand_idxs = np.random.permutation(1000)
train_idxs = rand_idxs[:800] # choosing the first 8000 as training set
val_idxs = rand_idxs[800:] # choosing last 2000 as validation set
train_paths = paths_subset[train_idxs]
val_paths = paths_subset[val_idxs]
print(len(train_paths), len(val_paths))

800 200


In [ ]:
_, axes = plt.subplots(4, 4, figsize=(10, 10))
for ax, img_path in zip(axes.flatten(), train_paths):
    ax.imshow(Image.open(img_path))
    ax.axis("off")

In [4]:
train_dl = make_dataloaders(paths=train_paths, split='train', color_space=COLOR_SPACE)
val_dl = make_dataloaders(paths=val_paths, split='val', color_space=COLOR_SPACE)

data = next(iter(train_dl))
known_channels, unknown_channels_ = data['known_channel'], data['unknown_channels']
print(known_channels.shape, unknown_channels_.shape)
print(len(train_dl), len(val_dl))

C:\Anaconda\lib\site-packages\torchvision\transforms\transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


torch.Size([8, 1, 256, 256]) torch.Size([8, 2, 256, 256])
100 25


In [25]:
net_G = build_res_unet(n_input=1, n_output=2, size=256)
opt = torch.optim.Adam(net_G.parameters(), lr=1e-4)
criterion = torch.nn.L1Loss()        
pretrain_generator(net_G, train_dl, opt, criterion, 1, device)


Epoch 1/1
L1 Loss: 0.08785


In [26]:
torch.save(net_G.state_dict(), "res18-unet.pt")

In [5]:
net_G = build_res_unet(n_input=1, n_output=2, size=256)
net_G.load_state_dict(torch.load("res18-unet.pt", map_location=device))

<All keys matched successfully>

In [ ]:
model = MainModel(net_G=net_G)
train_model(model, train_dl, val_dl, COLOR_SPACE, 2, 10)

In [ ]:
from visualize import visualize

loaded_model = MainModel()
_, loss_meter_dict = load_model(model_path, loaded_model)
for i, data in enumerate(iter(val_dl)):
    visualize(loaded_model, data, COLOR_SPACE, save=False)
    if i == 4:
        break